In [20]:
import os
import sys

ROOT_CRAWL = os.path.join(os.path.dirname("__file__"), "..", "crawl")


# Load data

In [21]:
import json

with open(os.path.join(ROOT_CRAWL, "list_stocks.json"), "r") as f:
    list_stocks = json.load(f)
print([x["CategoryName"] for x in list_stocks["category"]])

# symbols = list(filter(lambda x: x["CategoryName"] == "Ngân Hàng", list_stocks["category"]))[0][
#     "ListStockCode"
# ]
symbols = ["VCI"]


['Danh mục quan tâm', 'BDS', 'BDS - KCN', 'Bán lẻ', 'Bảo Hiểm', 'Cao su', 'Chứng Khoán', 'Cảng Biển & Vận Tải', 'Dầu khí', 'Dệt may', 'Ngân Hàng', 'Phân Bón & Hóa chất', 'Thép', 'Thủy sản', 'Trụ', 'Xây dựng', 'Điện gió', 'Đường, thực phẩm, gạo', 'Đầu tư công']


In [22]:
import pandas as pd

balancesheet = pd.read_csv(os.path.join(ROOT_CRAWL, "balancesheet.csv"), encoding="utf-8")
balancesheet["quarter_start_time"] = balancesheet.apply(
    lambda row: (pd.Period(freq="Q", year=row["year"], quarter=row["quarter"])).start_time,
    axis=1,
)
balancesheet["quarter_end_time"] = balancesheet.apply(
    lambda row: (pd.Period(freq="Q", year=row["year"], quarter=row["quarter"])).end_time, axis=1
)
balancesheet.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5154 entries, 0 to 5153
Data columns (total 39 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   ticker                  5154 non-null   object        
 1   quarter                 5154 non-null   int64         
 2   year                    5154 non-null   int64         
 3   shortAsset              4444 non-null   float64       
 4   cash                    5154 non-null   int64         
 5   shortInvest             4444 non-null   float64       
 6   shortReceivable         4444 non-null   float64       
 7   inventory               4444 non-null   float64       
 8   longAsset               4444 non-null   float64       
 9   fixedAsset              5154 non-null   int64         
 10  asset                   5154 non-null   int64         
 11  debt                    5154 non-null   int64         
 12  shortDebt               4444 non-null   float64 

In [23]:
financialratio = pd.read_csv(os.path.join(ROOT_CRAWL, "financialratio.csv"), encoding="utf-8")
financialratio["quarter_start_time"] = financialratio.apply(
    lambda row: (pd.Period(freq="Q", year=row["year"], quarter=row["quarter"])).start_time,
    axis=1,
)
financialratio["quarter_end_time"] = financialratio.apply(
    lambda row: (pd.Period(freq="Q", year=row["year"], quarter=row["quarter"])).end_time, axis=1
)
financialratio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4465 entries, 0 to 4464
Data columns (total 61 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   ticker                   4465 non-null   object        
 1   quarter                  4465 non-null   int64         
 2   year                     4465 non-null   int64         
 3   priceToEarning           2868 non-null   float64       
 4   priceToBook              2866 non-null   float64       
 5   valueBeforeEbitda        3585 non-null   float64       
 6   dividend                 0 non-null      float64       
 7   roe                      4258 non-null   float64       
 8   roa                      4458 non-null   float64       
 9   daysReceivable           3666 non-null   float64       
 10  daysInventory            3539 non-null   float64       
 11  daysPayable              3855 non-null   float64       
 12  ebitOnInterest           3232 non-

In [24]:
incomestatement = pd.read_csv(os.path.join(ROOT_CRAWL, "incomestatement.csv"), encoding="utf-8")
incomestatement["quarter_start_time"] = incomestatement.apply(
    lambda row: (pd.Period(freq="Q", year=row["year"], quarter=row["quarter"])).start_time,
    axis=1,
)
incomestatement["quarter_end_time"] = incomestatement.apply(
    lambda row: (pd.Period(freq="Q", year=row["year"], quarter=row["quarter"])).end_time, axis=1
)
incomestatement.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5397 entries, 0 to 5396
Data columns (total 26 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   ticker                          5397 non-null   object        
 1   quarter                         5397 non-null   int64         
 2   year                            5397 non-null   int64         
 3   revenue                         5397 non-null   int64         
 4   yearRevenueGrowth               4668 non-null   float64       
 5   quarterRevenueGrowth            4967 non-null   float64       
 6   costOfGoodSold                  4677 non-null   float64       
 7   grossProfit                     4677 non-null   float64       
 8   operationExpense                5296 non-null   float64       
 9   operationProfit                 5296 non-null   float64       
 10  yearOperationProfitGrowth       4080 non-null   float64       
 11  quar

# Merge all data

In [25]:
balancesheet_financialratio = balancesheet.merge(
    financialratio,
    left_on=["ticker", "quarter", "year", "quarter_start_time", "quarter_end_time"],
    right_on=["ticker", "quarter", "year", "quarter_start_time", "quarter_end_time"],
)
balancesheet_financialratio_incomestatement = balancesheet_financialratio.merge(
    incomestatement,
    left_on=["ticker", "quarter", "year", "quarter_start_time", "quarter_end_time"],
    right_on=["ticker", "quarter", "year", "quarter_start_time", "quarter_end_time"],
)
balancesheet_financialratio_incomestatement.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 4463 entries, 0 to 4462
Columns: 116 entries, ticker to ebitda
dtypes: datetime64[ns](2), float64(99), int64(14), object(1)
memory usage: 4.0+ MB


# Plot by group

In [26]:
balancesheet_financialratio_incomestatement[
    (balancesheet_financialratio_incomestatement["ticker"] == "HPG")
    & (balancesheet_financialratio_incomestatement["year"] == 2022)
    & (balancesheet_financialratio_incomestatement["quarter"] == 3)
].to_json()

'{"ticker":{"2626":"HPG"},"quarter":{"2626":3},"year":{"2626":2022},"shortAsset":{"2626":96871.0},"cash":{"2626":11881},"shortInvest":{"2626":27030.0},"shortReceivable":{"2626":12008.0},"inventory":{"2626":43880.0},"longAsset":{"2626":86935.0},"fixedAsset":{"2626":71725},"asset":{"2626":183805},"debt":{"2626":85730},"shortDebt":{"2626":52871.0},"longDebt":{"2626":12629.0},"equity":{"2626":98075},"capital":{"2626":58148},"centralBankDeposit":{"2626":null},"otherBankDeposit":{"2626":null},"otherBankLoan":{"2626":null},"stockInvest":{"2626":null},"customerLoan":{"2626":null},"badLoan":{"2626":null},"provision":{"2626":null},"netCustomerLoan":{"2626":null},"otherAsset":{"2626":null},"otherBankCredit":{"2626":null},"oweOtherBank":{"2626":null},"oweCentralBank":{"2626":null},"valuablePaper":{"2626":null},"payableInterest":{"2626":null},"receivableInterest":{"2626":null},"deposit":{"2626":null},"otherDebt":{"2626":740},"fund":{"2626":null},"unDistributedIncome":{"2626":0.0},"minorShareHolderP

In [27]:
balancesheet_financialratio_incomestatement_per_group = balancesheet_financialratio_incomestatement[
    balancesheet_financialratio_incomestatement["ticker"].isin(symbols)
]


## Earning Per Share

In [28]:
import plotly.express as px

fig = px.line(balancesheet_financialratio_incomestatement_per_group, x="quarter_end_time", y="earningPerShare", color="ticker", title='EPS')
fig.show()

## P/E

In [29]:
import seaborn as sns
import plotly.express as px

fig = px.line(
    balancesheet_financialratio_incomestatement_per_group[
        (balancesheet_financialratio_incomestatement_per_group["priceToEarning"] >= -10)
        & (balancesheet_financialratio_incomestatement_per_group["priceToEarning"] <= 50)
    ],
    x="quarter_end_time",
    y="priceToEarning",
    color="ticker",
    title="P/E",
)
fig.show()


## Book-Value Per Share

In [30]:
import seaborn as sns
import plotly.express as px

fig = px.line(
    balancesheet_financialratio_incomestatement_per_group,
    x="quarter_end_time",
    y="bookValuePerShare",
    color="ticker",
    title="Book-Value Per Share",
)
fig.show()


## P/B

In [31]:
import seaborn as sns
import plotly.express as px

fig = px.line(
    balancesheet_financialratio_incomestatement_per_group,
    x="quarter_end_time",
    y="priceToBook",
    color="ticker",
    title="P/B",
)
fig.show()


## ROA (Return on Assets)

In [32]:
import seaborn as sns
import plotly.express as px

fig = px.line(
    balancesheet_financialratio_incomestatement_per_group,
    x="quarter_end_time",
    y="roa",
    color="ticker",
    title="ROA",
)
fig.layout.yaxis.tickformat = ',.0%'
fig.show()

## ROE (Return on Equity)

In [33]:
import seaborn as sns
import plotly.express as px

fig = px.line(
    balancesheet_financialratio_incomestatement_per_group,
    x="quarter_end_time",
    y="roe",
    color="ticker",
    title="ROE",
)
fig.layout.yaxis.tickformat = ',.0%'
fig.show()

## Biên LNST(%)

In [34]:
import seaborn as sns
import plotly.express as px

fig = px.line(
    balancesheet_financialratio_incomestatement_per_group,
    x="quarter_end_time",
    y="postTaxMargin",
    color="ticker",
    title="Biên LNST(%)",
)
fig.layout.yaxis.tickformat = ',.0%'
fig.show()

## Biên LN gộp(%)

In [35]:
import seaborn as sns
import plotly.express as px

fig = px.line(
    balancesheet_financialratio_incomestatement_per_group,
    x="quarter_end_time",
    y="grossProfitMargin",
    color="ticker",
    title="Biên LN gộp",
)
fig.layout.yaxis.tickformat = ',.0%'
fig.show()

## DAR - Vay/Tài sản

In [36]:
import seaborn as sns
import plotly.express as px

fig = px.line(
    balancesheet_financialratio_incomestatement_per_group,
    x="quarter_end_time",
    y="debtOnAsset",
    color="ticker",
    title="Vay/Tài sản",
)
fig.show()

## ROIC

In [37]:
balancesheet_financialratio_incomestatement_per_group[
    "roic"
] = balancesheet_financialratio_incomestatement_per_group["postTaxProfit"] / (
    balancesheet_financialratio_incomestatement_per_group["equity"]
    + balancesheet_financialratio_incomestatement_per_group["longDebt"]
)

/tmp/ipykernel_1801722/3244879522.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [38]:
import seaborn as sns
import plotly.express as px

fig = px.line(
    balancesheet_financialratio_incomestatement_per_group,
    x="quarter_end_time",
    y="roic",
    color="ticker",
    title="ROIC",
)
fig.layout.yaxis.tickformat = ',.0%'
fig.show()